In [1]:
import nbdev

In [2]:
#| default_exp store

In [1]:
#|export
from __future__ import annotations
from typing import List, Callable, TypeVar,  Generic, Sequence, Union, Optional

### Types

In [12]:
#| export

T = TypeVar("T")

_Subscriber = Callable[[T], None]

_Unsubscriber = Callable[[], None]

_Updater = Callable[[T], T]

class Subscriber:
    def __init__(self, fn: _Subscriber, on_error: Optional[Callable], on_complete: Optional[Callable]) -> None:
        self.fn = fn
        self._on_error_handler = on_error
        self._on_complete_handler = on_complete

    def __call__(self, value: T) -> None:
        self.fn(value)

    def __eq__(self, other: _Subscriber) -> bool:
        return self.fn == other.fn

    def __hash__(self) -> int:
        return hash(self.fn)
    
    def on_next(self, value: _T_in) -> None:
        """Notify the observer of a new element in the sequence."""
        if not self.is_stopped:
            self._on_next_core(value)

    def _on_next_core(self, value: _T_in) -> None:
        """For Subclassing purpose. This method is called by `on_next()`
        method until the observer is stopped.
        """
        self._handler_on_next(value)

    def on_error(self, error: Exception) -> None:
        """Notify the observer that an exception has occurred.
        Args:
            error: The error that occurred.
        """

        if not self.is_stopped:
            self.is_stopped = True
            self._on_error_core(error)

class Readable(Generic[T]):
    def __init__(self, value: T) -> None:
        self.value = value
        self.subscribers: List[_Subscriber] = [] # callback list

    def __getattr__(self, name):
        if name in self.__dict__: return self.__dict__[name]
        return getattr(self.value, name)
    
    def subscribe(self, subscriber: _Subscriber) -> _Unsubscriber:
        self.subscribers.append(subscriber)
        subscriber(self.value)
        return lambda: self.subscribers.remove(subscriber) 

class Writable(Readable, Generic[T]):
    def set(self, new_value: T) -> None:
        if new_value != self.value: self.value = new_value
        for subscriber in self.subscriber_queue: subscriber(self.value)

    def update(self, fn: _Updater[T]) -> None: self.set(fn(self.value))


In [8]:
# #| export

# def writable(value: T) -> Writable[T]:
#     """ Create a writable store with a given value that allows both updating and reading by subscription."""

    
#     def set(new_value: T) -> None:
#         nonlocal value
#         if new_value != value: value = new_value
#         for subscriber in subscriber_queue: subscriber(value)

#     def update(fn: Updater[T]) -> None: set(fn(value))

#     def subscribe(subscriber: Subscriber[T]) -> Unsubscriber:
#         subscriber_queue.append(subscriber)
#         subscriber(value)

#         def unsubscribe() -> None:
#             subscriber_queue.remove(subscriber)

#         return unsubscribe

#     ret = Writable() # type: ignore
#     ret.set = set # type: ignore
#     ret.update = update # type: ignore
#     ret.subscribe = subscribe

#     return ret

In [6]:
# #| export

# def readable(value: T) -> Readable[T]:
#     ret = Readable()
#     ret.subscribe = writable(value).subscribe
#     return ret

### Export

In [10]:
nbdev.nbdev_export()

In [ ]:
class Store:
    def __init__(self, value):
        self.value = value
        self.subscribers = []
    def subscribe(self, subscriber):
        self.subscribers.append(subscriber)
        subscriber(self.value)
        return lambda: self.subscribers.remove(subscriber)
    def set(self, value):
        if value != self.value:
            self.value = value
            for subscriber in self.subscribers:
                subscriber(value)
    def update(self, updater):
        self.set(updater(self.value))